In [3]:
import redis, json, time, operator

## 建立連線與向資料庫倒入資料

In [3]:
'''
建立連線，輸入相關參數即可
'''
def connect4Input(host, port, db, password):
    connection = redis.Redis(host = host, port = port, db = db, password = password)
    print 'Connection: %s' % True
    return connection

'''
呼叫pipeline實體化，使用transation，
故使用mulit這個方法，如果執行上沒有錯誤，
接著就透過execute這個方法送出資料
如果發生錯誤，繼續使用continue使資料倒進資料庫不會中斷
'''
def ready2Input(connection, result):
    for tourAgency in result.keys():
        with connection.pipeline(transaction = True) as pipe:
            tourAgencyCount = 0
            try:
                pipe.watch(tourAgency)
                tourAgencyCount += 1
                pipe.multi()
                connection.hmset(tourAgency, result[tourAgency])
                pipe.execute()
            except redis.WatchError:
                continue

## 執行

In [9]:
path = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\totalAtriclesResult\final_result_proportion.json'
with open(path) as dataRead:
    start = time.time()
    result = json.load(dataRead)
    connection = connect4Input(host = '140.115.236.201', port = 6379, db = 0, password = '123456')
    ready2Input(connection, result)
    end = time.time()
    print 'Data is finished to input in Redis!!'
    print 'Time Cost: %f sec.' % (end - start)

Connection: True
Data is finished to input in Redis!!
Time Cost: 4.090000 sec.


## Query

In [5]:
'''
取得連線，並回傳連線物件
'''
def connect4Query(host, port, db, password):
    myServer = redis.Redis(host = host, port = port, db = db, password = password)
    print 'Connection: %s' % True
    return myServer

'''
外部來的參數有3個，第1個是取得連線物件，
第2個是要查詢的旅行社名稱，
第3個則是要取前N個的數量。
接著將取出的word_count做資料型態轉換，
Redis中，所有key,Value都會變成Str，
所以，取出時做排序前，必須先轉換成Int，
最後回傳前N個值(包在List中)
'''
def changeType2SortData(myServer, tourAgency, topn):
    new_result = list()
    result = myServer.hscan_iter(tourAgency)
    for count_word, word_count in result: #eval(word_count) == int(word_count)
        new_result.append((count_word.decode('utf-8'), eval(word_count)))
    new_result = sorted(new_result, key = operator.itemgetter(1), reverse = True)
    return new_result[0:topn]

'''
實際執行的方法，
取得連線後，再輸入要查詢的名稱，
為了方便前端人員的作業，
所以再將原本tuple的型態，
一一取出，轉換成k,v的樣子。
並且依序放進list中。
'''
def start():
    textRank = list()
    tourAgency = raw_input(u'請輸入你要查詢的旅行社: ')
    myServer = connect4Query(host = '36.236.48.175', port = 6379, db = 0, password = None)
    new_result = changeType2SortData(myServer, tourAgency, 15)
    for count_word, word_count in new_result:
        textRank.append({'emotion_word':count_word, 'word_count':word_count})
    return textRank

In [6]:
start()

請輸入你要查詢的旅行社: ftsTour
Connection: True


[{'emotion_word': u'\u6700\u597d', 'word_count': 0.513356},
 {'emotion_word': u'\u4e0d\u932f', 'word_count': 0.420518},
 {'emotion_word': u'\u518d\u9ebb\u7169', 'word_count': 0.263851},
 {'emotion_word': u'\u5f88\u6eff\u610f', 'word_count': 0.165397},
 {'emotion_word': u'\u4fbf\u5b9c', 'word_count': 0.160793},
 {'emotion_word': u'\u5c08\u696d', 'word_count': 0.143383},
 {'emotion_word': u'\u53ea\u6709\u721b', 'word_count': 0.118177},
 {'emotion_word': u'\u6279\u8a55', 'word_count': 0.106895},
 {'emotion_word': u'\u6bd4\u8f03\u597d', 'word_count': 0.106855},
 {'emotion_word': u'\u5f88\u666e\u901a', 'word_count': 0.09749},
 {'emotion_word': u'\u5f88\u4e0d\u8ca0\u8cac\u4efb', 'word_count': 0.089012},
 {'emotion_word': u'\u4e0d\u5c08\u696d', 'word_count': 0.071512},
 {'emotion_word': u'\u5f88\u512a\u60e0', 'word_count': 0.067198},
 {'emotion_word': u'\u9084\u662f\u58de', 'word_count': 0.063218},
 {'emotion_word': u'\u62b1\u6028', 'word_count': 0.060265}]

In [17]:
help(redis)

Help on package redis:

NAME
    redis

FILE
    d:\program files\anaconda2\lib\site-packages\redis\__init__.py

PACKAGE CONTENTS
    _compat
    client
    connection
    exceptions
    lock
    sentinel
    utils

CLASSES
    __builtin__.object
        redis.client.StrictRedis
            redis.client.Redis
        redis.connection.Connection
            redis.connection.SSLConnection
            redis.connection.UnixDomainSocketConnection
        redis.connection.ConnectionPool
            redis.connection.BlockingConnectionPool
    exceptions.Exception(exceptions.BaseException)
        redis.exceptions.RedisError
            redis.exceptions.AuthenticationError
            redis.exceptions.ConnectionError
                redis.exceptions.BusyLoadingError
            redis.exceptions.DataError
            redis.exceptions.InvalidResponse
            redis.exceptions.PubSubError
            redis.exceptions.ResponseError
                redis.exceptions.ReadOnlyError
            redi